<a href="https://colab.research.google.com/github/Jaishree935/Image-Classification-using-CNN/blob/main/Image_Classification_using_basic_Hyperparameter_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification using CNN with the CIFAR-10 Dataset using **Basic Hyperparameter Tunning**

#Steps to be followed

1. Importing necessary libraries
2. Loading and pre-processing of the CIFAR-10 dataset.
3. Defining the class names for CIFAR-10 images.
4. Define a function to build the model
5. choose an optimizer and learning rate
6. Perform the Hyperparameter search
7. Get the best Hyperparameters
8. Build the model with the best Hyperparameters and train it
9. Plotting the training and validation accuracy and loss values

#Installation

In [ ]:
pip install keras_tuner

# Step 1: Importing the necessary Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras_tuner.tuners import RandomSearch
import matplotlib.pyplot as plt


# Step 2: Loading and pre-processing of the CIFAR-10 dataset.

In [ ]:
(train_images, train_labels),(test_images, test_labels) = datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


# Step 3:Defining the class names for CIFAR-10 images.

In [ ]:
class_name = ['airplane' , 'automobile' , 'bird' , 'cat' ,'deer' ,'dog' ,'frog' ,'horse','ship','truck']

# Step 4: Define a function to build the model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, models
import keras_tuner as kt  # or `import kerastuner as kt` if using older version

def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(hp.Int('conv1_units', min_value=32, max_value=128, step=16),
                            (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(hp.Int('conv2_units', min_value=32, max_value=128, step=16),
                            (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(hp.Int('conv3_units', min_value=32, max_value=128, step=16),
                            (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense_units', min_value=32, max_value=128, step=16), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(layers.Dense(10))

    # Choose an optimizer and learning rate
    optimizer = keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))

    model.compile(optimizer=optimizer,
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model


# Step 5: Define the Tuner

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='cifar10_tunning'
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Step 6: Perform the Hyperparameter search

In [ ]:
tuner.search(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

Trial 3 Complete [00h 26m 30s]
val_accuracy: 0.6295999884605408

Best val_accuracy So Far: 0.6295999884605408
Total elapsed time: 00h 55m 17s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
32                |128               |conv1_units
80                |96                |conv2_units
64                |128               |conv3_units
80                |112               |dense_units
0.4               |0.2               |dropout_rate
0.01              |0.001             |learning_rate

Epoch 1/5
 604/1563 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.1000 - loss: 5.0947

# Step 7: Get the best Hyperparameters

In [ ]:
best_hps= tuner.get_best_hyperparameters(num_trials=1)[0]

# Step 8: Build the model with the best Hyperparameters and train it

In [ ]:
model=tuner.hypermodel.build(best_hps)
history = model.fit(train_images, train_labels, epochs=10,validation_data=(test_images,test_labels))

# Step 9: Plotting the training and validation accuracy and loss values

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'],label='Training Accuracy')
plt.plot(history.history['val_accuracy'],label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim([0,1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.grid()


plt.subplot(1,2,2)
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim([0,1])
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.grid()

plt.show()
